In [1]:
import pandas as pd
import numpy as np
GBQ_PROJECT_ID = '620265099307'

#q = '''SELECT * FROM ebmdatalab.outlier_detection.chem_by_subpara_by_practice_juntoaug17
#WHERE 1=1'''
#df1 = pd.io.gbq.read_gbq(q, GBQ_PROJECT_ID, dialect='standard')
# rows: pct, chemical, subpara, num, denom, ratio (num and denom are items not quantity)
df1 = pd.read_csv('chem_by_subpara_by_practice_juntoaug17',dtype={'subpara': object}) # read from local csv as memory issues with pd.io

q2 = '''SELECT DISTINCT chemical, chemical_code from ebmdatalab.hscic.bnf'''
chem = pd.io.gbq.read_gbq(q2, GBQ_PROJECT_ID, dialect='standard',verbose=False)

q3 = '''SELECT DISTINCT subpara, subpara_code from ebmdatalab.hscic.bnf'''
subp = pd.io.gbq.read_gbq(q3, GBQ_PROJECT_ID, dialect='standard',verbose=False)


C:\ProgramData\Anaconda2\lib\site-packages\oauth2client\_helpers.py:255: UserWarning: Cannot access bigquery_credentials.dat: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))



Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/v2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&response_type=code&client_id=495642085510-k0tmvj2m941jhre2nbqka17vqpjfddtd.apps.googleusercontent.com&access_type=offline

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.


In [2]:
# need to flag where ccgs have not prescribed any items of the denominator in order to clean the data. 

# Step 1: amend the datafrome to include a line for every CCG and every chemical and subparagraph.

# list all subpara-chemical combinations 
a = df1[["subpara", "chemical"]].drop_duplicates()

#list all practices
b = df1[["pct","practice"]].drop_duplicates()

# cross join to make table of all CCGs and all subpara combinations 
a['tmp'] = 1
b['tmp'] = 1
c = b.merge(a, on="tmp").drop('tmp', axis=1) # 237,636 rows

# join to data - need to list every possible chemical against every CCG
data = c.merge(df1, how="left", on=["pct","practice","subpara","chemical"])  # 237,636 rows
data


# Step 2: identify those with zero subparas
# subpara totals by ccg
subpara = df1[["pct","practice","subpara","denom"]].groupby(["subpara","pct","practice"]).max().reset_index() # 42,917 rows

#list all possible subparagraphs and all ccgs
a2 = df1[["subpara"]].drop_duplicates()
a2['tmp'] = 1

# cross join to CCGs to make table of all CCGs and all subpara combinations 
c2 = b.merge(a2, on="tmp").drop('tmp', axis=1) # 56,097 rows

# join to subpara data by ccg to identify subparas prescribed by each ccg.  
d = c2.merge(subpara,how="left", on=["subpara","pct","practice"])

# for subparas never prescribed, replace NAs with zeros so that there is data present to indicate this
d = d.fillna(0)

# join back to original dataset
d2 = d.merge(data, how="left", on=["subpara","pct","practice"], suffixes=("_subpara",""))
# check how many have zero denominators:
# data.loc[(data["denom_subpara"]==0)] # 19,665 rows 

# exclude combinations where denominators are zero THEN replace NAs with 0:
data2 = d2.loc[(d2["denom_subpara"]!=0)]
data2 = data2.fillna(0)
#data2 = data2.set_index(['pct','chemical'])
data2.head(3)

,pct,practice,subpara,denom_subpara,chemical,num,denom,ratio
0,04V,C82041,0307000,44.0,0307000J0,44.0,44.0,1.0
1,04V,C82041,0307000,44.0,0307000K0,0.0,0.0,0.0
2,04V,C82041,0307000,44.0,0307000Q0,0.0,0.0,0.0


# National
- take ratios of chemical/subparagraph from above
- calculate the *national* mean ratio and standard deviation for each chemical

In [3]:
df2_nat = data2[["chemical", "ratio"]].groupby(["chemical"]).describe()
df2_nat.columns = df2_nat.columns.droplevel()
df2_nat.head(3)

,count,mean,std,min,25%,50%,75%,max
chemical,,,,,,,,
0101010C0,5624.0,0.035670,0.148797,0.0,0.0,0.000,0.0,1.0
0101010F0,5624.0,0.003186,0.044284,0.0,0.0,0.000,0.0,1.0
0101010G0,5624.0,0.414975,0.394887,0.0,0.0,0.375,0.8,1.0


- calculate the number of standard deviations each practice is from the national mean, for each chemical

In [4]:
output_nat = data2.merge(df2_nat[['mean','std']],how='left', left_on=['chemical'],right_index=True)

output_nat['stds_from_mean'] = (output_nat['ratio'] - output_nat['mean']) / output_nat['std']
output_nat = output_nat.dropna().sort_values('stds_from_mean',ascending =False)
output_nat = output_nat.merge(chem, how="left", left_on = "chemical",right_on="chemical_code",suffixes=(""," name"))
output_nat = output_nat.merge(subp, how="left", left_on = "subpara",right_on="subpara_code",suffixes=(""," name"))
output_nat = output_nat.drop(['subpara','denom_subpara','chemical'],axis=1)
output_nat = output_nat.set_index('practice')
#output_nat.head(3)

### More than usual

- filter out ones with small ratio

In [6]:
output_nat.loc[(output_nat.denom>200)
               &(output_nat['mean']>0.01)].sort_values('stds_from_mean',
                                                       ascending =False).head(5)

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
M85670,13P,222.0,347.0,0.639769,0.020097,0.027576,22.471677,Pantoprazole,0103050R0,Proton Pump Inhibitors,0103050
L84615,11M,425.0,701.0,0.606277,0.021632,0.029649,19.718936,Valsartan,0205052V0,Angiotensin-II Receptor Antagonists,0205052
C83022,99D,1537.0,2306.0,0.666522,0.017529,0.034551,18.783890,Telmisartan,0205052Q0,Angiotensin-II Receptor Antagonists,0205052
F81751,06Q,308.0,547.0,0.563071,0.021632,0.029649,18.261701,Valsartan,0205052V0,Angiotensin-II Receptor Antagonists,0205052
M85139,13P,579.0,1137.0,0.509235,0.020097,0.027576,17.737998,Pantoprazole,0103050R0,Proton Pump Inhibitors,0103050


### Less than usual

- filter out ones with small ratio

In [8]:
output_nat.loc[(output_nat.denom>200)
               &(output_nat['mean']<0.99)].sort_values('stds_from_mean',
                                                       ascending =True).head(5)

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
J81078,11J,126.0,236.0,0.533898,0.978408,0.041976,-10.589725,Doxazosin Mesilate,0205040D0,Alpha-Adrenoceptor Blocking Drugs,0205040
L82039,11N,130.0,233.0,0.557940,0.978408,0.041976,-10.016972,Doxazosin Mesilate,0205040D0,Alpha-Adrenoceptor Blocking Drugs,0205040
C83631,99D,235.0,365.0,0.643836,0.955130,0.031335,-9.934361,Colecalciferol,0906040G0,Vitamin D,0906040
B81095,03F,169.0,258.0,0.655039,0.962421,0.031150,-9.867778,Glucose Blood Testing Reagents,0601060D0,Diabetic Diagnostic & Monitoring Agents,0601060
M83121,05G,165.0,289.0,0.570934,0.978408,0.041976,-9.707403,Doxazosin Mesilate,0205040D0,Alpha-Adrenoceptor Blocking Drugs,0205040


## 99P
### More than usual

- filter out ones with small ratio

In [10]:
output_nat.loc[(output_nat.pct=='99P')
               &(output_nat.denom>200)
               &(output_nat['mean']>0.01)].sort_values('stds_from_mean',
                                                       ascending =False).head(5)

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
L83012,99P,293.0,478.0,0.612971,0.017529,0.034551,17.233945,Telmisartan,0205052Q0,Angiotensin-II Receptor Antagonists,0205052
L83643,99P,136.0,281.0,0.483986,0.049498,0.043056,10.091244,Nifedipine,0206020R0,Calcium-Channel Blockers,0206020
L83663,99P,164.0,561.0,0.292335,0.039735,0.025490,9.909595,Paroxetine Hydrochloride,0403030P0,Selective Serotonin Re-Uptake Inhibitors,0403030
L83134,99P,711.0,1087.0,0.654094,0.051798,0.074624,8.071075,Lercanidipine Hydrochloride,0206020L0,Calcium-Channel Blockers,0206020
L83023,99P,186.0,653.0,0.284839,0.017529,0.034551,7.736790,Telmisartan,0205052Q0,Angiotensin-II Receptor Antagonists,0205052


### Less than usual

- filter out ones with small ratio

In [12]:
output_nat.loc[(output_nat.pct=='99P')
               &(output_nat.denom>200)
               &(output_nat['mean']<0.99)].sort_values('stds_from_mean',
                                                       ascending =True).head(5)

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
L83112,99P,131.0,262.0,0.500000,0.884759,0.084858,-4.534128,Tamsulosin Hydrochloride,0704010U0,Drugs For Urinary Retention,0704010
L83673,99P,123.0,290.0,0.424138,0.805378,0.090115,-4.230574,Amitriptyline Hydrochloride,0403010B0,Tricyclic & Related Antidepressant Drugs,0403010
L83134,99P,312.0,1087.0,0.287029,0.708416,0.130218,-3.236014,Amlodipine,0206020A0,Calcium-Channel Blockers,0206020
L83136,99P,115.0,213.0,0.539906,0.822748,0.089521,-3.159482,Amoxicillin,0501013B0,Broad-Spectrum Penicillins,0501013
L83128,99P,33.0,265.0,0.124528,0.715778,0.189366,-3.122261,Tiotropium,0301020Q0,Antimuscarinic Bronchodilators,0301020


## Practices deviating most from national average for each chemical/subpara

- mean deviation in either direction (i.e. inverting negative values)

In [45]:
np.abs(output_nat.loc[(output_nat.denom>200)&(output_nat['mean']>0.01),['stds_from_mean']]).groupby(level=0).mean().sort_values(['stds_from_mean'],ascending=False).head(10)

,stds_from_mean
practice,
E87006,3.165891
C84602,2.754485
Y04925,2.356867
F84025,2.211170
E84696,2.183346
Y02045,2.103095
B81666,2.037823
Y02873,1.864375
C85628,1.861658


## Chemicals with the most deviating practices

In [52]:
output_nat.loc[(output_nat.denom>200)&(output_nat['mean']>0.01),['stds_from_mean','chemical_code','chemical name']].groupby('chemical_code').mean().sort_values(['stds_from_mean'],ascending=False).head(10)

,stds_from_mean
chemical_code,
1306020J0,8.294428
1404000H0,3.775838
1308010Y0,1.928783
0604012M0,1.928128
1203010Q0,1.752001
1302020D0,1.509387
0309020G0,1.226250
0208010D0,1.203144
0703050B0,0.938184


In [50]:
output_nat.loc[(output_nat.denom>200)&(output_nat['mean']>0.01),['stds_from_mean','chemical_code','chemical name']].groupby('chemical_code').mean().sort_values(['stds_from_mean'],ascending=True).head(10)

,stds_from_mean
chemical_code,
1306020C0,-8.277003
0604012P0,-1.309967
130801000,-1.111594
1302020E0,-1.023510
0703050A0,-0.938184
090607000,-0.786796
1301010I0,-0.597345
0702010F0,-0.583685
1308010W0,-0.578204


# Within CCGs
- take ratios of chemical/subparagraph from top
- calculate the *CCG level* mean ratio and standard deviation for each chemical
- *all* results from here have rows with small ratio filtered out

In [13]:
df2 = data2[["pct","chemical", "ratio"]].groupby(["pct","chemical"],sort=False).describe()
df2.columns = df2.columns.droplevel()
df2.head(3)

count      mean       std   min       25%       50%  75%   max
pct chemical                                                                 
00C 0101010C0   11.0  0.015455  0.037779  0.00  0.000000  0.000000  0.0  0.12
    0101010F0   11.0  0.000000  0.000000  0.00  0.000000  0.000000  0.0  0.00
    0101010G0   11.0  0.922104  0.136904  0.55  0.936538  0.965517  1.0  1.00

- calculate the number of standard deviations each practice is from the CCG level mean, for each chemical

In [14]:
output = data2.merge(df2[['mean','std']],how='left', left_on=['pct','chemical'],right_index=True)

output['stds_from_mean'] = (output['ratio'] - output['mean']) / output['std']
output = output.dropna().sort_values('stds_from_mean',ascending =False)
output = output.merge(chem, how="left", left_on = "chemical",right_on="chemical_code",suffixes=(""," name"))
output = output.merge(subp, how="left", left_on = "subpara",right_on="subpara_code",suffixes=(""," name"))
output = output.drop(['subpara','denom_subpara','chemical'],axis=1)
output = output.set_index('practice')
#output.head(3)

### More than usual

In [15]:
output.loc[(output.denom>200)
           &(output['mean']>0.01)].sort_values('stds_from_mean',
                                               ascending =False).head()

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
D81056,06H,436.0,525.0,0.830476,0.013463,0.085288,9.579452,Influenza,1404000H0,Vaccines And Antisera,1404000
L83024,99P,265.0,520.0,0.509615,0.010224,0.054215,9.211343,Influenza,1404000H0,Vaccines And Antisera,1404000
J81074,11J,1213.0,1510.0,0.803311,0.037695,0.085128,8.993761,Irbesartan,0205052I0,Angiotensin-II Receptor Antagonists,0205052
L83663,99P,164.0,561.0,0.292335,0.031725,0.029198,8.925740,Paroxetine Hydrochloride,0403030P0,Selective Serotonin Re-Uptake Inhibitors,0403030
D81021,06H,457.0,528.0,0.865530,0.032666,0.094390,8.823692,Glycopyrronium Bromide,0301020S0,Antimuscarinic Bronchodilators,0301020


### Less than usual

In [16]:
output.loc[(output.denom>200)
           &(output['mean']<0.99)].sort_values('stds_from_mean',
                                               ascending =True).head()

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
L84020,11M,205.0,374.0,0.548128,0.971112,0.056330,-7.509083,Gliclazide,0601021M0,Sulfonylureas,0601021
J81078,11J,183.0,368.0,0.497283,0.886661,0.060762,-6.408204,Alendronic Acid,0606020A0,Bisphosphonates and Other Drugs,0606020
N85038,12F,99.0,278.0,0.356115,0.956773,0.094338,-6.367063,Gliclazide,0601021M0,Sulfonylureas,0601021
J81647,11J,149.0,220.0,0.677273,0.938204,0.041694,-6.258211,Salbutamol,0301011R0,Selective Beta(2)-Agonists,0301011
L85044,11X,181.0,335.0,0.540299,0.923622,0.062176,-6.165172,Tamsulosin Hydrochloride,0704010U0,Drugs For Urinary Retention,0704010


## 99P
### More than usual

In [17]:
output.loc[(output.pct=='99P')
           &(output.denom>200)
           &(output['mean']>0.01)].sort_values('stds_from_mean',
                                               ascending =False).head()

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
L83024,99P,265.0,520.0,0.509615,0.010224,0.054215,9.211343,Influenza,1404000H0,Vaccines And Antisera,1404000
L83663,99P,164.0,561.0,0.292335,0.031725,0.029198,8.925740,Paroxetine Hydrochloride,0403030P0,Selective Serotonin Re-Uptake Inhibitors,0403030
L83012,99P,293.0,478.0,0.612971,0.018599,0.069770,8.519005,Telmisartan,0205052Q0,Angiotensin-II Receptor Antagonists,0205052
L83113,99P,98.0,230.0,0.426087,0.026975,0.053875,7.408123,Diethylamine Salicylate,1003020I0,"Rubefacients,Top NSAIDS,Capsaicin & Poul",1003020
L83073,99P,113.0,497.0,0.227364,0.016114,0.030845,6.848756,Glipizide,0601021P0,Sulfonylureas,0601021


### Less than usual

In [18]:
output.loc[(output.pct=='99P')
           &(output.denom>200)
           &(output['mean']<0.99)].sort_values('stds_from_mean',
                                               ascending =True).head()

,pct,num,denom,ratio,mean,std,stds_from_mean,chemical name,chemical_code,subpara name,subpara_code
practice,,,,,,,,,,,
L83073,99P,366.0,497.0,0.736419,0.970214,0.042183,-5.542384,Gliclazide,0601021M0,Sulfonylureas,0601021
L83673,99P,123.0,290.0,0.424138,0.813865,0.070350,-5.539809,Amitriptyline Hydrochloride,0403010B0,Tricyclic & Related Antidepressant Drugs,0403010
L83112,99P,131.0,262.0,0.500000,0.865584,0.073971,-4.942270,Tamsulosin Hydrochloride,0704010U0,Drugs For Urinary Retention,0704010
L83039,99P,300.0,421.0,0.712589,0.927178,0.046157,-4.649120,Furosemide,0202020L0,Loop Diuretics,0202020
L83107,99P,205.0,282.0,0.726950,0.927178,0.046157,-4.337980,Furosemide,0202020L0,Loop Diuretics,0202020


## Practices that deviate most from their CCG

In [46]:
np.abs(output.loc[(output.denom>200)&(output['mean']>0.01),['stds_from_mean']]).groupby(level=0).mean().sort_values(['stds_from_mean'],ascending=False).head(10)

,stds_from_mean
practice,
F84731,2.323890
B81666,2.224210
Y02045,2.146629
Y02873,2.112559
E84696,2.072997
E85672,2.027288
C84602,1.970829
Y00054,1.934354
E87006,1.867021


### 99P

In [49]:
np.abs(output.loc[(output.pct=='99P')&(output.denom>200)&(output['mean']>0.01),['stds_from_mean']]).groupby(level=0).mean().sort_values(['stds_from_mean'],ascending=False).head(10)

,stds_from_mean
practice,
L83673,1.460568
L83643,1.394315
L83012,1.113370
L83642,1.073722
L83080,1.002173
L83663,0.982917
L83651,0.980097
L83107,0.965890
L83023,0.919517
